## 创建会话
会话(Session)拥有并管理TensorFlow程序运行时的所有资源。所有计算完成之后需要关闭会话来回收资源，否则可能出现资源泄漏的问题。TensorFlow中使用会话的模式一般有两种：<br/>
- 需要明确调用会话生成函数和关闭函数。<br/>

In [ ]:
# 创建一个会话
sess = tf.Session()
# 使用会话得到计算结果
sess.run(result)
# 关闭会话并释放资源
sess.close()

使用该种模式在计算完成后，需要调用Session.close()来关闭会话并释放资源。然而，当程序异常而退出时，可能关闭会话函数不会被执行而导致资源泄漏。<br/>
- 通过Python上下文管理器来使用会话。当上下文管理器退出时候会自动释放所有资源。<br/>

In [ ]:
# 创建会话，并使用Python上下文管理器来管理会话。
with tf.Session() as sess:
    # 使用会话得到计算结果
    sess.run(result)
# 不需要再调用关闭会话资源函数

TensorFlow不会跟计算图一样自动生成默认的会话，而是需要手动指定。当默认会话被指定之后可以通过`tf.Tensor.eval()`来计算一个tensor的取值。

In [ ]:
# 通过设定默认会话计算张量的取值
sess = tf.Session()
with sess.as_default():
    print(result.eval())

In [ ]:
# 以下代码完成类似功能
sess = tf.Session()
# 以下两行代码具有相同功能
print(sess.run(result))
print(result.eval(session=sess))

在`交互式环境`下，可以直接使用`tf.InterActiveSession()`函数在交互式环境下直接构建默认会话。通过`InterActiveSession()`函数可以直接省略产生会话注册为默认会话的过程。

In [ ]:
sess = tf.InterActiveSession()
print(result.eval)
sess.close()

## 配置会话
无论使用哪种方式来创建会话都可以使用`ConfigProto Protocol Buffer`进行来配置。

In [ ]:
# ConfigProto配置会话
config = tf.ConfigProto(allow_soft_placement=True,
                       log_device_placement=True)
sess1 = tf.InterActiveSession(config=config)
sess2 = tf.Session(config=config)

ConfigProto可以配置并行的线程数，GPU分配策略和运算超时时间等参数。最常用的参数有两个，`allow_soft_placement`和`log_device_placement`。<br/>
- `allow_soft_placement`<br/>
布尔类型参数，当设置为`True`时，以下任意一个条件成立时，GPU上的运算可以放到CPU上进行。<br/>
1. 运算无法在GPU上执行。<br/>
2. 没有GPU资源。(比如运算被指定在第二个GPU上运行，机器只有一个GPU)。<br/>
3. 运算输入包含了对CPU计算结果的引用。<br/>
该参数默认值为`False`。但是为了使代码可移植性更强，在有GPU的环境下这个参数一般被设置为True。不同的GPU驱动版本可能对计算的支持有略微的区别，当前某些计算无法被GPU支持时，会被自动调整到CPU上，而不是报错。另外，可以让程序在不同GPU数量的机器上顺利运行。<br/>
- `log_device_placement`<br/>
布尔类型参数，当设置为True时，日志将记录每个节点被安排到哪个设备上以方便调试。而生产环境中将这个参数设置为False可减少日记量。